In [1]:
import xml.etree.ElementTree as ET
import duckdb
from datetime import datetime
from duckdb import ConstraintException, FatalException
from pathlib import Path
from tqdm import tqdm
import traceback

In [2]:
# Connect to sql database
con = duckdb.connect(database='../data/database/german-parliament.duckdb', read_only=False)

In [3]:
def create_db(reset_db=False):
    """
    Creates the necessary tables in the DuckDB database for storing plenary minutes.
    This function creates the 'party', 'speaker', and 'plenary_minute' tables, as well as sequences for party and speaker IDs.

    Args:
        reset_db (bool): If True, drops existing tables and sequences before creating new ones.
                          Defaults to False.

    Returns:
        None
    """
    if reset_db:
        con.execute("DROP TABLE IF EXISTS speech;")
        con.execute("DROP TABLE IF EXISTS speaker;")
        con.execute("DROP TABLE IF EXISTS party;")
        con.execute("DROP TABLE IF EXISTS plenary_minute;")
        con.execute("DROP TABLE IF EXISTS session;")


        con.execute("DROP SEQUENCE IF EXISTS party_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speaker_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS legislative_period_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS session_id_seq;")
        con.execute("DROP SEQUENCE IF EXISTS speech_id_seq;")

    # Create a sequence for party IDs to ensure unique IDs for each party. It's like an auto-incrementing primary key.
    con.execute("CREATE SEQUENCE IF NOT EXISTS party_id_seq START 1;")
    # Create the party which will be used to identify a party
    con.execute(
        """
        CREATE TABLE IF NOT EXISTS party (
            id INTEGER DEFAULT nextval('party_id_seq') PRIMARY KEY,
            -- If we insert a party we just use DEFAULT to get the next value from the sequence.
            name VARCHAR UNIQUE NOT NULL
        );
        """
    )

    # Create a sequence for speaker IDs to ensure unique IDs for each speaker.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speaker_id_seq START 1;")
    # Create the speaker which will be used to identify a speaker
    con.execute("""
        CREATE TABLE IF NOT EXISTS speaker (
            harmonised_id INTEGER DEFAULT nextval('speaker_id_seq') UNIQUE,
            -- harmonised_id is used to identify the speaker in the XML file of all periods. The speaker id provided by the Bundestag for periods >= 19 is not speaker unique!
            -- If we insert a speaker we just use DEFAULT to get the next value from the sequence.
            name VARCHAR NOT NULL,
            party_id INTEGER REFERENCES party(id) NOT NULL,
            PRIMARY KEY (name, party_id)
        );
        """
    )

    # Create a sequence for legislative periods to ensure unique IDs for each legislative period.
    con.execute("CREATE SEQUENCE IF NOT EXISTS legislative_period_id_seq START 1;")
    # Create the session table to store information about legislative periods and sessions.
    con.execute("""
        CREATE TABLE IF NOT EXISTS session (
            id INTEGER DEFAULT nextval('legislative_period_id_seq') UNIQUE NOT NULL,
            legislative_period INTEGER NOT NULL,
            session_no INTEGER NOT NULL,
            date DATE NOT NULL,
            primary key (legislative_period, session_no)
        );
        """
    )

    # Create a sequence for plenary minute IDs to ensure unique IDs for each plenary minute.
    con.execute("CREATE SEQUENCE IF NOT EXISTS speech_id_seq START 1;")
    con.execute("""
        CREATE TABLE IF NOT EXISTS speech (
            id VARCHAR DEFAULT nextval('speech_id_seq') PRIMARY KEY,
            -- id is a unique identifier for each speech, generated from the sequence for speeches for periods < 19.
            -- For newer periods the bundestag provides a unique id for each speech. Those begin with "ID"
            session_id INTEGER REFERENCES session(id) NOT NULL,
            speaker_harmonised_id INTEGER REFERENCES speaker(harmonised_id), -- we can use harmonised_id, since this id is available for all periods
            role VARCHAR, -- Role of the speaker, e.g. 'mp'
            position VARCHAR, -- Position of the speaker, e.g. 'Präsident', 'Alterspräsident', etc. | In newer periods this is named Rolle!
            content TEXT NOT NULL
        );
        """
    )
    con.commit()

In [4]:
def parse_xml(file_path):
    """
    Parses an XML file and returns the root element.

    Args:
        file_path (str): The path to the XML file.

    Returns:
        xml.etree.ElementTree.Element: The root element of the parsed XML tree,
                                         or None if parsing fails.
    """
    try:
        tree = ET.parse(file_path)
        root = tree.getroot()
        return root
    except ET.ParseError as e:
        print(f"Error parsing XML file: {e}")
        return None
    except FileNotFoundError:
        print(f"Error: File not found at {file_path}")
        return None

In [126]:
def insert_session(legislative_period, session, date):
    """ Inserts session information into the session table.

    Args:
        meta_data (xml.etree.ElementTree.Element): The title statement element containing legislative period and session information, as well as the date.

    Raises:
        Exception: If the titleStmt is None or if the publicationStmt is not found in the XML file.

    Returns:
        None
    """
    # print("------------------------------------")
    # print(f"Legislative Period: {legislative_period}, Session: {session}")
    # print(f"Publication Date: {date}")#
    # print("------------------------------------\n")

    # Insert session into the session table(

    sql = """
        INSERT INTO session (legislative_period, session_no, date)
        VALUES (?, ?,?)
        ON CONFLICT DO NOTHING;
    """

    params = (
        legislative_period,
        session,
        date
    )
    con.execute(sql, params)

    # con.execute(f"""
    #     INSERT INTO session (id, legislative_period, session_no, date)
    #     VALUES (DEFAULT, '{legislative_period}', '{session}', '{date}');
    # """)


def insert_party(attributes):
    # Insert party into the party table if it does not exist. The on conflict do nothing clause ensures that our key constraints aren't violated
    sql = "INSERT INTO party (name) VALUES (?) ON CONFLICT DO NOTHING;"
    con.execute(sql, (attributes["party"],))


def insert_speaker(attributes):
    sql = """
            INSERT INTO speaker (name, party_id)
            VALUES (
                ?,                                    -- speaker’s full name
                (SELECT id FROM party WHERE name = ?) -- look-up party once, safely
            )
            ON CONFLICT DO NOTHING;
            """

    con.execute(sql, (attributes["name"], attributes["party"]))




def insert_speech(speech, attributes, legislative_period, session, speech_id=None):
    # Insert speech into the speech table
    # print(attributes)
    speech_string = ET.tostring(speech, encoding='unicode') if speech is not None else ''

    party_id = con.execute(f"""
                    SELECT id FROM party WHERE name = '{attributes['party']}';
                """).fetchone()[0]
    speaker_id = con.execute(f"""
                    SELECT harmonised_id FROM speaker WHERE name = '{attributes['name']}' AND party_id = {party_id};
                """).fetchone()[0]
    session_id = con.execute(f"""
                    SELECT id FROM session WHERE legislative_period = '{legislative_period}' AND session_no = '{session}';
                 """).fetchone()[0]

    speaker_role = attributes.get("role")
    speaker_role = None if speaker_role == 'NA' else speaker_role  # Convert 'NA' to None for SQL NULL
    speaker_position = attributes.get("position")
    speaker_position = None if speaker_position == 'NA' else speaker_position  # Convert 'NA' to None for SQL NULL
    # values to bind — order must match the placeholders
    if speech_id is None:
        # If we have no speech_id, we know that this is a speech from a period < 19 and we can use our own, automatically generating id.
        sql = """
        INSERT INTO speech
               (session_id, speaker_harmonised_id, role, position, content)
        VALUES (?, ?, ?, ?, ?)
        ON CONFLICT DO NOTHING;
        """
        params = (
        session_id,
        speaker_id,
        speaker_role,
        speaker_position,
        speech_string
        )

        con.execute(sql, params)
    else:
        # If we have a speech_id, we know that this is a speech from a period >= 19 and we can insert the new_id speech id instead of using our proprietary id.
        sql = """
        INSERT INTO speech
               (id, session_id, speaker_harmonised_id, role, position, content)
        VALUES (?, ?, ?, ?, ?, ?)
        ON CONFLICT DO NOTHING;
        """
        params = (
            speech_id,
            session_id,
            speaker_id,
            speaker_role,
            speaker_position,
            speech_string
        )
        con.execute(sql, params)

def get_speaker_attributes(speaker) -> dict:
    """
    Extracts speaker attributes from the speaker data.

    Args:
        speaker_data (xml.etree.ElementTree.Element): The speaker data element.

    Returns:
        dict: A dictionary containing the speaker's name, party, and ID.
    """
    speaker_data = speaker.find('name')
    # Extract the speaker's first and last name @todo maybe get rid of the title or save it in another column!
    speaker_title = speaker_data.find('titel')
    speaker_title = "" if speaker_title is None else speaker_title.text
    # Check if first name exists
    speaker_first_name = speaker_data.find('vorname')
    speaker_first_name = speaker_first_name.text if speaker_first_name is not None else None
    # Check if last name exist
    speaker_last_name = speaker_data.find('nachname')
    speaker_last_name = speaker_last_name.text if speaker_last_name is not None else None
    # @todo strip the name and get rid of \n
    speaker_full_name = None
    if speaker_first_name is None and speaker_last_name is None:
        #sometimes speeches are broken. See speech: ID196901400
        speaker_full_name = "NA"
    else:
        speaker_full_name = speaker_title + " " + speaker_first_name + " " + speaker_last_name
    # Extract the speakers party
    speaker_party = speaker_data.find('fraktion')
    if speaker_party is not None:
        speaker_party = speaker_party.text
    else:
        speaker_party = 'NA' # Use 'NA' if no party is found

    # Get the speaker position, e.g. "Alterspräsident" 
    speaker_position = speaker_data.find('rolle')
    if speaker_position is not None:
        speaker_position = speaker_position.find("rolle_lang").text
    else:
        speaker_position = None
    
    # Return the speaker attributes as a dictionary
    return {"party": speaker_party, "name": speaker_full_name, "position": speaker_position}


def parse_old_plenary_minute(root):
    """ Parses the plenary minute XML root element and extracts speeches, speakers, and parties and inserts them into the database.

    Args:
        root (xml.etree.ElementTree.Element): The root element of the XML tree containing the data of a single plenary minute.

    Raises:
        Exception: If the XML root element is None.

    Returns:
        None

    """
    if root is not None:
        # Get the meta data information, i.e. date, session, etc.
        meta_data = root.find('teiHeader').find('fileDesc')
        
        titleStmt = meta_data.find('titleStmt')
        publicationStmt = meta_data.find('publicationStmt')
        
        legislative_period = titleStmt.find('legislativePeriod').text
        session = titleStmt.find('sessionNo').text
        date = publicationStmt.find('date').text

        # Insert session entry into the db
        insert_session(legislative_period, session, date)
        #Get the actual content of the plenary minute
        plenary_minute = root.find('text').find('body').findall('div')
        speeches = []
        # Get all 'div' elements in the 'body' of the 'text'. Those are the elements that contain the actual plenary minute content.
        for div in plenary_minute:
            speeches.extend(div.findall('sp'))

        for speech in speeches:
            # Get attributes of the speech element
            attributes = speech.attrib
            # Insert party of speaker into db
            insert_party(attributes)
            # Insert speaker (politician) into db
            insert_speaker(attributes)
            # Insert the speech into the db
            insert_speech(speech, attributes, legislative_period, session)
    else:
        raise Exception("Missing XML root element.")

def parse_new_plenary_minute(root):
    if root is not None:
        # Get metadata
        kopfdaten = root.find("vorspann").find("kopfdaten")
        # Get period and session no
        plenarprotokoll_nummer = kopfdaten.find("plenarprotokoll-nummer")
        legislative_period = plenarprotokoll_nummer.find("wahlperiode").text
        session_number_tmp = plenarprotokoll_nummer.find("sitzungsnr").text
        # Sometime the editors of the xml added (neu) -> maybe if the original xml was faulty? 
        session_number = session_number_tmp.replace(" (neu)", "") 
        # Get the date
        raw_date = kopfdaten.find("veranstaltungsdaten").find('datum').attrib.get("date")
        date = datetime.strptime(raw_date, "%d.%m.%Y")
        # Insert the session into the database
        insert_session(legislative_period, session_number, date) #@ todo this will be the existing function
        # Get the session data where the speeches are stored
        tagesordnungspunkte = root.find("sitzungsverlauf").findall("tagesordnungspunkt")
        # Iterate over each Tagesordnungspunkt (agenda item)
        for top in tagesordnungspunkte:
            speeches = top.findall('rede')
            for speech in speeches:
                # This will extract the speaker data and feeds it into the database.
                speech_id = speech.attrib.get('id')
                # Finds the section where the speech of the speaker begins, this avoid comments from other before the speech. Then it get's the speaker's information
                speaker = speech.find(".//p[@klasse='redner']").find('redner')
                # try:
                attributes = get_speaker_attributes(speaker)
                insert_party(attributes) # passed as list to ensure compatibility with the insert function @todo existing function
                insert_speaker(attributes) #@ todo existing function
                insert_speech(speech, attributes, legislative_period, session_number, speech_id) #@todo existing function
                # except AttributeError as e:
                #     print(ET.tostring(speech, encoding='unicode'), "\n\n")
                #     traceback.print_tb(e.__traceback__)
                #     con.close()

def parse_legislative_period(path, period):
    print(f"Parsing legislative period at path: {path}")
    # We materialise the generator once, to obtain all entries, so we can call len() on it
    files = list(Path(path).rglob("*"))
    # tqdm is used to create a nice loading bar for progress
    for file in tqdm(files, total=len(files), desc="Processing files"):
        if file.suffix == ".xml":
            # print(f"Parsing file: {file}")
            root = parse_xml(file)
            # Depending on the period we parse the XML differently, since the structure of the XML changed after period 18.
            if period <= 18:
                parse_old_plenary_minute(root)
            elif period > 18:
                parse_new_plenary_minute(root)
    print(f"Finished parsing legislative period at path: {path}")



def main():
    # Create database tables
    create_db(False)
    # Parse each period between period 1 and 21
    for period in range(1, 22):
        # 02:d adjusts the number to be two digits, e.g. 01, 02, ..., 10, 11, ...
        try:
            con.execute("BEGIN TRANSACTION;")
            parse_legislative_period(f'../data/plenary_minutes/wahlperiode_{period:02d}', period)
            con.execute("COMMIT;")
        except FatalException as e:
            con.execute("ROLLBACK;")
            print(f"Error processing legislative period {period}: {e}")


In [127]:
if __name__ == "__main__":
    main()

# Always close connection to ensure that we won't have problems reconnecting to the db!
con.close()

Parsing legislative period at path: ../data/plenary_minutes/wahlperiode_19


Processing files:  31%|███       | 74/239 [00:28<01:02,  2.63it/s]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

Processing files: 100%|██████████| 239/239 [01:59<00:00,  2.00it/s]


Finished parsing legislative period at path: ../data/plenary_minutes/wahlperiode_19
Parsing legislative period at path: ../data/plenary_minutes/wahlperiode_20


Processing files: 100%|██████████| 214/214 [01:22<00:00,  2.59it/s]


Finished parsing legislative period at path: ../data/plenary_minutes/wahlperiode_20
Parsing legislative period at path: ../data/plenary_minutes/wahlperiode_21


Processing files: 100%|██████████| 6/6 [00:01<00:00,  3.79it/s]


Finished parsing legislative period at path: ../data/plenary_minutes/wahlperiode_21


In [148]:
con.close()

In [153]:
con.execute("select speech.id, role, position, content, harmonised_id, speaker.name, party.name from speech join speaker on speaker.harmonised_id = speech.speaker_harmonised_id JOIN party on speaker.party_id = party.id offset 1000000").fetchone()[3]

'<sp who="Oliver Kaczmarek" parliamentary_group="SPD" role="mp" position="NA" who_original="Oliver Kaczmarek" party="SPD" name="Oliver Kaczmarek">\n          <speaker>Oliver Kaczmarek (SPD):</speaker>\n          <p>Herr Präsident! Meine Damen und Herren! Liebe Kolleginnen und Kollegen! Wenn wir die Inklusionsdiskussion vor Ort betrachten, dann sehen wir, dass der Bereich Bildung oft eine herausgehobene Bedeutung hat. Dabei werden viele Sorgen, Hoffnungen, aber eben auch Ängste deutlich: Welche Kita ist die richtige? Welche Schule ist die richtige? Was passiert nach der Schule? Wie finde ich einen Studienplatz? Aber es geht auch um die Fragen: Reicht meine Ausbildung als Lehrer, um der inklusiven Bildung gerecht zu werden? Wer hilft meinem Kind? Was passiert mit meinem Arbeitsplatz?</p>\n          <p>Das alles zeigt aus meiner Sicht die Dimension und Herausforderung für unser Bildungswesen. Inklusion betont Individualität. Inklusion meint: Verschiedenheit ist der Normalfall. Genau das b

In [ ]:
con.execute("select * from speaker join party on speaker.party_id = party.id ;").fetchdf()

In [ ]:
print('hi')